In [0]:
!pip install eli5


     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
y_true = df['prices_amountmin']
price_log_mean=np.expm1(np.mean(np.log1p(y_true)))
y_pred = [price_log_mean] * y_true.shape[0]

mean_absolute_error(y_true, y_pred)

58.03539547661041

In [0]:
df.brand.value_counts()

Nike              1709
PUMA               574
Ralph Lauren       524
New Balance        361
VANS               356
                  ... 
Co7de                1
Savane               1
GAP                  1
zagone studios       1
New Era              1
Name: brand, Length: 1856, dtype: int64

In [0]:
mean_price = np.mean(df['prices_amountmin'])
mean_price

93.53952188183217

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X=df[ feats ].values
  y=df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['prices_amountmax'])

(-6.798523977405554, 1.4724661004940744)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [0]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['prices_amountmax'],model)

(-5.811679264840806, 1.5431507652053014)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
from ast import literal_eval

def parse_features(x):
  output_dict ={}
  
  if str(x) == 'nan':
    return output_dict
	
  features = literal_eval(x.replace('\\"','"'))
  
  for item in features:
    key = item['key'].lower().strip()
    value= item['value'][0].lower().strip()
    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)


476

In [0]:
from tqdm import tqdm_notebook

def get_name_feat(key):
	return 'feat_'+ key

for key in tqdm_notebook(keys):
	df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key]  if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_manufacturer part number', 'feat_reinforced knee',
       'feat_display technology', 'feat_chain/necklace length (in.)',
       'feat_best sellers rank', 'feat_flame resistant',
       'feat_amazonbestsellersrank', 'feat_country////region of manufacture',
       'feat_is dark sky-compliant', 'feat_lens technology'],
      dtype='object', length=526)

In [0]:
for key in keys:
  df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

df[df.brand != df.feat_brand ][['brand','feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
df[df.brand != df.feat_brand ].shape

(9434, 531)

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']

model= RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feats,model)

(-57.103455281769286, 4.242816377164084)

In [0]:
from eli5.sklearn import PermutationImportance

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)



Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [0]:

feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_material_cat']

model= RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feats,model)

(-57.1079495610026, 4.2018887077121025)

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [0]:
df['brand'].value_counts()

nike                 1777
puma                  609
ralph lauren          526
vans                  386
new balance           371
                     ... 
firefly                 1
marc new york           1
peterluckyi             1
shoe goo                1
wisconsin badgers       1
Name: brand, Length: 1732, dtype: int64

In [0]:
df['brand'].value_counts(normalize=True)

nike                 0.097210
puma                 0.033315
ralph lauren         0.028775
vans                 0.021116
new balance          0.020295
                       ...   
firefly              0.000055
marc new york        0.000055
peterluckyi          0.000055
shoe goo             0.000055
wisconsin badgers    0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df [ df['brand']  == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df [ df['brand']  == 'nike' ].features_parsed.sample(5).values

array([{'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/volt', 'casual & dress shoe style': 'sandals', 'manufacturer part number': '705474070', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'style': 'running, cross training', 'gender': 'mens', 'condition': 'new with box'},
       {'style': 'basketball shoes', 'pattern': 'solid', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': "men's shoes", 'features': 'durable material ,�� comfortable fit', 'brand': 'nike'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/beige', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '366177018', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [0]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
mens                   4
infant                 4
boys'                  3
women ,�� unisex       2
youth                  2
men||women             2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
	df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
feat_cats = [x for x in df.columns if 'cat' in x]
feat_cats

['categories',
 'brand_cat',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_catalog',
 'feat_clothing category',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_fabrication',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_golf shoe type_cat',
 'feat_band material_cat',
 'feat_number of wheels_cat',
 'feat_chronograph_cat',
 'feat_footwear type_cat',
 'feat_lined_cat',
 'feat_assembled product weight_cat',
 'feat_pocket_cat',
 'feat_height_cat',
 'feat_is lined_cat',
 'feat_bag size range_cat',
 'feat_материал_cat',
 'feat_diamond clarity_cat',
 'feat_count_cat',
 'feat_boxed-product dimensions_cat',
 'feat_foot arch_cat',
 'feat_digital camera_cat',
 'feat_product type_cat',
 'feat_clothing type_cat',
 'feat_jacket length_cat',
 'feat_used with equipment type_cat',
 'feat_con

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
#feats += feat_cats
#feats = list(set(feats))
feats=[ 'feat_fit to_cat',
 'feat_material type_cat',
 'feat_age range_cat',
 'feat_conflict_cat',
 'feat_shoe closure_cat',
 'feat_mpn_cat',
 'feat_gold karat_cat',
 'feat_toe type_cat',
 'feat_contains batteries_cat',
 'feat_band length (in.)_cat',
 'feat_band width_cat',
 'feat_is waterproof_cat',
 'feat_clothing product type_cat',
 'feat_year made_cat',
 'feat_safety apparel product type_cat',
 'feat_accessory type_cat',
 'feat_hooded_cat',
 'feat_product #_cat',
 'feat_occasion_cat',
 'feat_watch power source_cat',
 'feat_face shape_cat',
 'feat_profession_cat',
 'feat_sku_cat',
 'feat_metal type_cat',
 'feat_features & fastening_cat',
 'feat_reinforced pockets_cat',
 'feat_chest size_cat',
 'feat_shoe fastening type_cat',
 'feat_type_cat',
 'feat_style code_cat',
 'feat_fabric care instructions_cat',
 'feat_fabric type_cat',
 'feat_sports team_cat',
 'feat_primary shelf id_cat',
 'feat_watch style_cat',
 'feat_pattern or design_cat',
 'feat_inv_cat',
 'feat_garment care_cat',
 'feat_reinforced knee_cat',
 'feat_display technology_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_best sellers rank_cat',
 'feat_flame resistant_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_country////region of manufacture_cat',
 'feat_is dark sky-compliant_cat',
 'feat_lens technology_cat']

model= RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(feats,model)

(-63.1012524764234, 3.4102022905542873)

In [0]:
X = df[feats].values
y = df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.0292 ± 0.0023,feat_band width_cat
0.0106 ± 0.0017,feat_occasion_cat
0.0073 ± 0.0015,feat_face shape_cat
0.0045 ± 0.0007,feat_metal type_cat
0.0042 ± 0.0012,feat_gold karat_cat
0.0025 ± 0.0004,feat_watch style_cat
0.0022 ± 0.0002,feat_type_cat
0.0015 ± 0.0003,feat_mpn_cat
0.0007 ± 0.0003,feat_sports team_cat
0.0007 ± 0.0003,feat_band length (in.)_cat


In [55]:
!pwd

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [56]:
!git add matrix_one/day_5.ipynb

fatal: pathspec 'matrix_one/day_5.ipynb' did not match any files


In [57]:
!pwd

/content/drive/My Drive/Colab Notebooks/dw_matrix
